In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout,BatchNormalization, GlobalAveragePooling2D, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from PIL import Image, ImageChops, ImageEnhance
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import cv2
from keras.utils import to_categorical
from keras.applications import ResNet50, VGG16, MobileNet, DenseNet121
from keras.models import Model, load_model, Sequential
from keras.regularizers import l2
import random
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dir_path = '/kaggle/input/comofod/CoMoFoD_small_v2'

comofod_data = {'image_id': [],'image_path': [],'label': []}
  
for img in os.listdir(dir_path):
    if 'F' in img:
        temp_path = os.path.join(dir_path, img)
        comofod_data['image_path'].append(temp_path)
        comofod_data['label'].append('fake')
        comofod_data['image_id'].append(img)
    if 'O' in img:
        temp_path = os.path.join(dir_path, img)
        comofod_data['image_path'].append(temp_path)
        comofod_data['label'].append('real')
        comofod_data['image_id'].append(img)
        
label_encoder = LabelEncoder()
comofod_data = pd.DataFrame(comofod_data)
comofod_data['label'] = label_encoder.fit_transform(comofod_data['label'])
comofod_data = comofod_data.sample(frac=1, random_state=random.seed(42)).reset_index(drop=True)
comofod_data.head()

In [ ]:
train, test = train_test_split(comofod_data[['image_path', 'label']], test_size=0.2, random_state=42)

In [ ]:
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256, 256))

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_eq = cv2.equalizeHist(img_gray)

    img_denoised = cv2.fastNlMeansDenoising(img_eq, None, h=10, templateWindowSize=7, searchWindowSize=21)
    img_color_corrected = cv2.cvtColor(img_denoised, cv2.COLOR_GRAY2BGR)
    img_color_corrected = cv2.cvtColor(img_color_corrected, cv2.COLOR_BGR2HSV)
    img_color_corrected[:, :, 1] = img_color_corrected[:, :, 1] * 1.2
    img_color_corrected[:, :, 2] = img_color_corrected[:, :, 2] * 0.8
    img_color_corrected = cv2.cvtColor(img_color_corrected, cv2.COLOR_HSV2BGR)

    return img_color_corrected

In [ ]:
class Dataset:
    def __init__(self, data, shuffle=True, augment=False):
        self.image_paths = data['image_path'].tolist()
        self.labels = to_categorical(data['label'].astype(np.int32).tolist())
        self.shuffle = shuffle
        self.augment = augment
        self._shuffle_data()

    def _shuffle_data(self):
        if self.shuffle:
            indices = np.random.permutation(len(self.image_paths))
            self.image_paths = [self.image_paths[i] for i in indices]
            self.labels = self.labels[indices]

    def _augment_image(self, img):
        datagen = ImageDataGenerator(
            rotation_range=20,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest'
        )

        img = datagen.random_transform(img)
        return img

    def __getitem__(self, i):
        img = preprocess_image(self.image_paths[i])
        if self.augment:
            img = self._augment_image(img)

        img = img.astype(np.float32)/255
        label = self.labels[i]
        return img, label

    def __len__(self):
        return len(self.image_paths)

    def on_epoch_end(self):
        self._shuffle_data()

class Dataloader(tf.keras.utils.Sequence):
    def __init__(self, dataset, batch_size=1, shuffle=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.dataset))

    def __getitem__(self, i):
        start = i * self.batch_size
        stop = (i + 1) * self.batch_size
        data = [self.dataset[j] for j in range(start, stop)]

        batch = [np.stack(samples, axis=0) for samples in zip(*data)]
        input_img_batch = batch[0]
        label_batch = batch[1]

        return input_img_batch, label_batch

    def __len__(self):
        return len(self.indexes) // self.batch_size

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indexes)


In [ ]:
train_dataset = Dataset(train, shuffle = True, augment=True)
train_dataloader = Dataloader(train_dataset, batch_size=32, shuffle=True)

test_dataset = Dataset(test)
test_dataloader = Dataloader(test_dataset, batch_size=32, shuffle=True)

In [ ]:
# Vanilla Model

vanillaModel = Sequential()
vanillaModel.add(Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)))
vanillaModel.add(BatchNormalization())
vanillaModel.add(MaxPooling2D((2, 2)))
vanillaModel.add(Conv2D(64, (3, 3), activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(MaxPooling2D((2, 2)))
vanillaModel.add(Conv2D(128, (3, 3), activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(MaxPooling2D((2, 2)))
vanillaModel.add(Conv2D(256, (3, 3), activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(MaxPooling2D((2, 2)))
vanillaModel.add(GlobalAveragePooling2D())
vanillaModel.add(Dense(256, activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(Dropout(0.5))
vanillaModel.add(Dense(128, activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(Dropout(0.5))
vanillaModel.add(Dense(64, activation='relu'))
vanillaModel.add(BatchNormalization())
vanillaModel.add(Dropout(0.5))
vanillaModel.add(Dense(2, activation='softmax'))

vanillaModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
vanillaModel.summary()

In [ ]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.001)

vanillaModel.fit(train_dataloader, validation_data=test_dataloader, epochs=25, callbacks=[early_stop, learning_rate])


In [ ]:
vanillaModel.save('vanillaModel.h5')

In [ ]:
# ResNet50 Model

resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
resNetModel = GlobalAveragePooling2D()(resnet.output)
resNetModel = Dense(1024, activation='relu')(resNetModel)
resNetModel = Dense(1024, activation='relu')(resNetModel)
resNetModel = Dense(512, activation='relu')(resNetModel)
output = Dense(2, activation='softmax')(resNetModel)

resNetModel = Model(resnet.inputs, output)

resNetModel.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

resNetModel.summary()

In [ ]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.001)

resNetModel.fit(train_dataloader, validation_data=test_dataloader, epochs=25, callbacks=[early_stop, learning_rate])

In [ ]:
resNetModel.save('NewresNet50.h5')

In [ ]:
# MobilNet

mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in mobilenet.layers:
    layer.trainable = False
    
mobileNetModel = GlobalAveragePooling2D()(mobilenet.output)
mobileNetModel = Dense(1024, activation='relu')(mobileNetModel)
mobileNetModel = Dense(1024, activation='relu')(mobileNetModel)
mobileNetModel = Dense(512, activation='relu')(mobileNetModel)
output_mobile = Dense(2, activation='softmax')(mobileNetModel)

mobilenetModel = Model(mobilenet.inputs, output_mobile)

mobilenetModel.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

mobilenetModel.summary()

In [ ]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.001)

mobilenetModel.fit(train_dataloader, validation_data=test_dataloader, epochs=25, callbacks=[early_stop, learning_rate])

In [ ]:
mobilenetModel.save('mobileNet.h5')

In [ ]:
# Vgg16

vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in vgg16.layers:
    layer.trainable = False
    
vgg16Model = GlobalAveragePooling2D()(vgg16.output)
vgg16Model = Dense(1024, activation='relu')(vgg16Model)
vgg16Model = Dense(1024, activation='relu')(vgg16Model)
vgg16Model = Dense(512, activation='relu')(vgg16Model)
output_vgg16 = Dense(2, activation='softmax')(vgg16Model)

vgg16Model = Model(vgg16.inputs, output_vgg16)

vgg16Model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

vgg16Model.summary()

In [ ]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.001)

vgg16Model.fit(train_dataloader, validation_data=test_dataloader, epochs=25, callbacks=[early_stop, learning_rate])

In [ ]:
vgg16Model.save('vgg16.h5')

In [ ]:
# DenseNet 121

denseNet = DenseNet121(input_shape=(256, 256, 3), include_top=False, weights='imagenet')

for layer in denseNet.layers:
    layer.trainable = False

denseNetModel = denseNet.output
denseNetModel = GlobalAveragePooling2D()(denseNetModel)
denseNetModel = Dense(1024, activation='relu')(denseNetModel)
denseNetModel = Dense(1024, activation='relu')(denseNetModel)
denseNetModel = Dense(512, activation='relu')(denseNetModel)
denseNetModel = BatchNormalization()(denseNetModel)
denseNetModel = Dropout(0.5)(denseNetModel)

output_layer = Dense(2, activation='softmax')(denseNetModel)

denseNetModel = Model(inputs=denseNet.input, outputs=output_layer)

denseNetModel.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

denseNetModel.summary()


In [ ]:
early_stop = EarlyStopping(patience=10, restore_best_weights=True)
learning_rate = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1, patience=5, min_lr=0.001)

denseNetModel.fit(train_dataloader, validation_data=test_dataloader, epochs=25, callbacks=[early_stop, learning_rate])

In [ ]:
denseNetModel.save('denseNetModel.h5')